In [2]:
import numpy as np
import scipy.linalg as linalg

# Black Scholes Formula
$
{\frac {\partial V}{\partial t}}+{\frac {1}{2}}\sigma ^{2}S^{2}{\frac {\partial ^{2}V}{\partial S^{2}}}+rS{\frac {\partial V}{\partial S}}-rV=0
$

In [63]:
mm = 10
nn = 10

tt = np.arange(10)

print(np.arange(10))

for i in tt:
    # print(i)
    # print(i+1)
    pass

grid = np.zeros(shape=(mm+1, nn+1))

profit = 49.33 * np.exp(-0.1 * (1 - 0.01))
print(profit)
profit = 49.33 * np.exp(-0.1 * (1 - 0.99))
print(profit)

[0 1 2 3 4 5 6 7 8 9]
44.68028778680165
49.28069465678039


# Finite Difference Class
## Discretization
- S is discretized by N Steps, $\Delta s = S_{max}/N$, then it can be splited as $\{s_0, s_1, s_2, \ldots, s_M\}$ or $\{0\Delta s, 1\Delta s, 2\Delta s, \ldots, (M-1)\Delta s, M\Delta s\}$
- T is discretized by M Steps, $\Delta t = T/M$, then it can be splited as $\{t_0, t_1, t_2, \ldots, t_M\}$ or $\{0\Delta t, 1\Delta t, 2\Delta t, \ldots, (N-1)\Delta t, N\Delta t\}$
- Using i to represent each space(price) step, $\{i_0, i_1, i_2, \ldots, j_{M-1}, i_M\}$
- Using j to represent each time step: $\{j_0, j_1, j_2, \ldots, j_{N-1}, j_N\}$

In [33]:
class FiniteDifferences(object):

    def __init__(self, S0, K, r, T, sigma, Smax, M, N,
                 is_call=True):
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.sigma = sigma
        self.Smax = Smax
        self.M, self.N = int(M), int(N)  # Ensure M&N are integers
        self.is_call = is_call
        
        # calculate the discretization size of the space(price) and time
        self.dS = Smax / float(self.M)
        self.dt = T / float(self.N)
        
        # create index list
        self.i_values = np.arange(self.M)
        self.j_values = np.arange(self.N)
        
        # save all the option price at each step into a 2-D matrix
        self.grid = np.zeros(shape=(self.M+1, self.N+1))
        
        # manipulate the asset price when the discretize step
        self.boundary_conds = np.linspace(0, Smax, self.M+1)

    def _setup_boundary_conditions_(self):
        pass

    def _setup_coefficients_(self):
        pass

    def _traverse_grid_(self):
        """  Iterate the grid backwards in time """
        pass

    def _interpolate_(self):
        """
        Use piecewise linear interpolation on the initial
        grid column to get the closest price at S0.
        """
        return np.interp(self.S0, 
                         self.boundary_conds,
                         self.grid[:, 0])
        
    def price(self):
        self._setup_boundary_conditions_()
        self._setup_coefficients_()
        self._traverse_grid_()
        return (self._interpolate_(), self.grid)
    
    def print_grid(self):
        possible_prices = {}
        for j in self.j_values:
            tmp  = []
            for i in range(self.M):
                tmp.append(self.grid[i, j])
                pass
            possible_prices[j] = tmp
        return possible_prices
    
    def get_info(self):
        infos = {}
        for j in self.j_values:
            tmp  = []
            for i in range(self.M):
                info = {'asset': i*self.dS, 'time': j*self.dt, 'option': self.grid[i, j]}
                tmp.append(info)
                pass
            infos[j] = tmp
        return infos

# The Explicit Method
## Difference approximation method
- Forward Difference approximation with respect to t.
- First-order Central Difference approximation with respect to S.
- Second-Order Symmetric Central Difference approximation with respect to S.

In [34]:
""" Explicit method of Finite Differences """
class FDExplicitEu(FiniteDifferences):

    def _setup_boundary_conditions_(self):
        if self.is_call:
            # Terminal/exercise when asset price S > strike price K
            self.grid[:, -1] = np.maximum(self.boundary_conds - self.K, 0)
            # Terminal at the end of the option contract
            self.grid[-1, :-1] = (self.Smax - self.K) * np.exp(-self.r *
                                                               self.dt *
                                                               (self.N - self.j_values))
        else:
            # Terminal/exercise when asset price S > strike price K
            self.grid[:, -1] = np.maximum(self.K-self.boundary_conds, 0)
            # Terminal at the end of the option contract
            self.grid[0, :-1] = (self.K - self.Smax) * np.exp(-self.r * 
                                                              self.dt * 
                                                              (self.N - self.j_values))

    def _setup_coefficients_(self):
        self.a = 0.5 * self.dt * ((self.sigma**2) *
                                  (self.i_values**2) - 
                                  self.r * self.i_values)
        self.b = 1 - self.dt * ((self.sigma**2) * 
                                (self.i_values**2) + 
                                self.r)
        self.c = 0.5 * self.dt * ((self.sigma**2) * 
                                  (self.i_values**2) + 
                                  self.r*self.i_values)

    def _traverse_grid_(self):
        for j in reversed(self.j_values):
            for i in range(self.M)[2:]:
                self.grid[i, j] = self.a[i] * self.grid[i-1, j+1] + self.b[i] * self.grid[i, j+1] + self.c[i] * self.grid[i+1, j+1] 

In [59]:
option = FDExplicitEu(50, 50, 0.1, 1.0, 0.4, 100, 150, 10000, True)
print(option.price()[0])
prices = option.print_grid()
print(len(prices))

info = option.get_info()
print(len(info))
print(info[100][-1])
print(info[5000][-1])
print(info[9999][-1])

# for k in option.j_values:
#     print(len(prices[k]))
#     pass

9.90909108638459
10000
10000
{'asset': 99.33333333333333, 'time': 0.01, 'option': 44.885865360560274}
{'asset': 99.33333333333333, 'time': 0.5, 'option': 47.077458637693795}
{'asset': 99.33333333333333, 'time': 0.9999, 'option': 49.33383333333333}


In [58]:
option1 = FDExplicitEu(50, 50, 0.1, 1.0, 0.4, 100, 150, 10000, False)
print(option1.price()[0])
prices = option1.print_grid()
print(len(prices))

info1 = option1.get_info()
print(len(info1))
print(info1[100][10])
print(info1[5000][10])
print(info1[9999][10])

5.399096286197699
10000
10000
{'asset': 6.666666666666666, 'time': 0.01, 'option': 38.61975602626809}
{'asset': 6.666666666666666, 'time': 0.5, 'option': 40.894783190659595}
{'asset': 6.666666666666666, 'time': 0.9999, 'option': 43.33283333333334}
